In [75]:
import os
import shutil
import argparse
import pandas as pd
import numpy as np
from tqdm import trange
# from iou import *

In [76]:
def IoU(bbox, gt):
    """
    :param bbox: (n, 4)
    :param gt: (m, 4)
    :return: (n, m)
    numpy 广播机制 从后向前对齐。 维度为1 的可以重复等价为任意维度
    eg: (4,3,2)   (3,2)  (3,2)会扩充为(4,3,2)
        (4,1,2)   (3,2) (4,1,2) 扩充为(4, 3, 2)  (3, 2)扩充为(4, 3,2) 扩充的方法为重复
    广播会在numpy的函数 如sum, maximun等函数中进行
    pytorch同理。
    扩充维度的方法：
    eg: a  a.shape: (3,2)  a[:, None, :] a.shape: (3, 1, 2) None 对应的维度相当于newaxis
    """
    lt = np.maximum(bbox[:, :2], gt[:, :2])  # left_top (x, y)
    #print(lt.shape)
    rb = np.minimum(bbox[:, 2:], gt[:, 2:])  # right_bottom (x, y)
    wh = np.maximum(rb - lt, 0)                # inter_area (w, h)
    inter_areas = wh[:, 0] * wh[:, 1]        
    print(inter_areas.shape)
    box_areas = (bbox[:, 2] - bbox[:, 0]) * (bbox[:, 3] - bbox[:, 1])
    # print(box_areas.shape)
    gt_areas = (gt[:, 2] - gt[:, 0]) * (gt[:, 3] - gt[:, 1])
    # print(gt_areas.shape)
    IoU = inter_areas / (box_areas + gt_areas - inter_areas)
    return IoU

In [77]:
dataset_names = ["fanzhuan","qiehuan1"]

def single_video_iou(algo,dataset_name,index):
    
    bbox_path = '{}/{}_combined/{}-{}_combined.csv'.format(algo, dataset_name,dataset_name,index)
    bbox_csv = pd.read_csv(bbox_path, header= 0)
    len_bbox_csv = len(bbox_csv)
    print("{}th bbox {} length read:".format(index,algo), len_bbox_csv)
    bbox_csv["x2"] = bbox_csv["x"]+bbox_csv["w"]
    bbox_csv["y2"] = bbox_csv["y"] + bbox_csv["h"]
    bbox_df = bbox_csv[["x","y","x2","y2"]]
    #print(bbox_df.head())

    gt_path = 'SeqTrack/{}_combined/{}-{}_combined.csv'.format(dataset_name,dataset_name,index)
    gt_csv = pd.read_csv(gt_path,header = 0)
    len_gt_csv = len(gt_csv)
    print("{}th ground truth {} length read:".format(index, "SeqTrack"), len_gt_csv)
    gt_csv['x2'] = gt_csv['x']+gt_csv['w']
    gt_csv["y2"] = gt_csv["y"] + gt_csv["h"]
    gt_df = gt_csv[["x","y","x2","y2"]]
    #print(gt_df.head())

    if len_bbox_csv != len_gt_csv:
        print("{}th length diff: ".format(index),len_bbox_csv-len_gt_csv)
    else:
        bbox_values = bbox_df.values
        gt_values = gt_df.values

        print("bbox:", bbox_values.shape)
        print("gt:", gt_values.shape)
        
        result_iou = np.mean(IoU(bbox_values,gt_values))
        print("IOU:",IoU(bbox_values,gt_values))
        return result_iou
        

single_video_iou("HCAT","qiehuan1",2)



2th bbox HCAT length read: 861
2th ground truth SeqTrack length read: 861
bbox: (861, 4)
gt: (861, 4)
(861,)
(861,)
IOU: [1.         0.9168885  0.92518419 0.91210981 0.92156381 0.91546592
 0.92737533 0.886429   0.92092245 0.9380121  0.8911374  0.91825192
 0.92068444 0.93560808 0.92589563 0.89275068 0.90700152 0.87858179
 0.88520917 0.87007626 0.8848609  0.85663718 0.90281975 0.86074725
 0.88267099 0.87292663 0.87395971 0.87314522 0.86884033 0.94048573
 0.89083662 0.88049475 0.87960086 0.94410519 0.86456746 0.90666283
 0.87557907 0.94079433 0.87663397 0.88221724 0.90940418 0.89239613
 0.87903269 0.92272739 0.90187418 0.92975171 0.87962124 0.86494545
 0.88489962 0.87614688 0.92274881 0.8631691  0.87990205 0.89940523
 0.8830827  0.90375337 0.87266001 0.90619818 0.87547102 0.93339477
 0.84578801 0.91171912 0.89679158 0.89610732 0.87983337 0.88619637
 0.8843485  0.89328936 0.87386867 0.91554381 0.87612684 0.90687016
 0.93930629 0.89906597 0.90862043 0.92618799 0.89065451 0.94779086
 0.85692

0.44878613694593383

In [78]:
def dataset_iou(algo,dataset_name):
    ious = []
    path = '{}/{}_combined/'.format(algo,dataset_name)
    files = os.listdir(path)

    dataset_len = len(files)- 1 
    for i in trange(dataset_len):
        iou = single_video_iou(algo,dataset_name,i)
        ious.append(iou)
    
    return ious



In [79]:
print(dataset_iou("HCAT","fanzhuan"))

 35%|███▌      | 7/20 [00:00<00:00, 67.64it/s]

0th bbox HCAT length read: 1272
0th ground truth SeqTrack length read: 1272
bbox: (1272, 4)
gt: (1272, 4)
(1272,)
(1272,)
IOU: [1.         0.89458853 0.88896275 ... 0.89664485 0.8722387  0.89148584]
1th bbox HCAT length read: 906
1th ground truth SeqTrack length read: 906
bbox: (906, 4)
gt: (906, 4)
(906,)
(906,)
IOU: [1.         0.90381847 0.91166508 0.93378314 0.8858366  0.92121695
 0.88642958 0.89685627 0.88751187 0.87836261 0.90979016 0.9126915
 0.9081989  0.92990629 0.89084205 0.91748672 0.9057715  0.91297581
 0.9218459  0.92714019 0.92543424 0.9176824  0.91777459 0.95236077
 0.92907659 0.92071655 0.93367236 0.92259739 0.94607225 0.95389784
 0.93316008 0.9592209  0.94351139 0.95915375 0.92881152 0.93262937
 0.92269065 0.92378553 0.93614971 0.94385775 0.94883603 0.89518313
 0.92696133 0.91371202 0.94396265 0.90892539 0.93932114 0.92169779
 0.91887258 0.90639001 0.92303675 0.91205451 0.91757981 0.90947087
 0.90715167 0.90527586 0.90047773 0.91552992 0.90466904 0.90382494
 0.8803097 

 75%|███████▌  | 15/20 [00:00<00:00, 74.44it/s]

10th bbox HCAT length read: 845
10th ground truth SeqTrack length read: 845
bbox: (845, 4)
gt: (845, 4)
(845,)
(845,)
IOU: [1.         0.76659202 0.64220007 0.71446566 0.63794704 0.65137322
 0.7153548  0.79564281 0.62434618 0.77219363 0.66446809 0.72263915
 0.72089524 0.6848498  0.62329232 0.65945023 0.60025548 0.6468961
 0.6327377  0.60096179 0.72749131 0.58414453 0.6852776  0.64711378
 0.62411332 0.65996527 0.61109877 0.58699358 0.60169385 0.62539506
 0.60810682 0.69370559 0.63420345 0.72851183 0.64510347 0.65026809
 0.67512311 0.65791916 0.60981841 0.65150318 0.67618991 0.57973103
 0.67805093 0.61148627 0.54482772 0.54079978 0.61492887 0.56037731
 0.55752733 0.61895148 0.52544901 0.59949034 0.58619529 0.6021549
 0.6627535  0.60026532 0.6140134  0.62435523 0.63061153 0.63779277
 0.58816385 0.58187666 0.66571952 0.55292153 0.67666426 0.66842083
 0.62015736 0.72956945 0.63113418 0.77014026 0.71504349 0.70505552
 0.76590869 0.70990896 0.81296705 0.67629163 0.73921801 0.67061003
 0.67198

100%|██████████| 20/20 [00:00<00:00, 76.63it/s]

15th ground truth SeqTrack length read: 773
bbox: (773, 4)
gt: (773, 4)
(773,)
(773,)
IOU: [1.         0.89001387 0.84925148 0.85674841 0.87253436 0.85852181
 0.8754354  0.86399547 0.85020363 0.8773417  0.86190142 0.86170611
 0.86076984 0.85430186 0.87673622 0.85037526 0.87318955 0.89359469
 0.86156409 0.84266392 0.88249347 0.89177214 0.88506424 0.86972501
 0.8942068  0.90220964 0.88378268 0.89276087 0.88638625 0.88093489
 0.88764761 0.88433517 0.89548033 0.89165944 0.89031834 0.88483182
 0.89465351 0.87614631 0.88428882 0.88055413 0.86943692 0.88385367
 0.88052312 0.87362674 0.86250663 0.87594631 0.85868377 0.86006337
 0.87519976 0.88890067 0.85387685 0.88209896 0.88249652 0.90178833
 0.87674875 0.89690507 0.88753668 0.88183773 0.86610011 0.89954525
 0.88689726 0.90393163 0.86456924 0.90107417 0.90318235 0.90851985
 0.91471319 0.9088334  0.89890453 0.89036507 0.90953584 0.90277931
 0.90405651 0.88655153 0.90740022 0.91412203 0.91989335 0.92820845
 0.91400486 0.90610741 0.88561353 0.90

In [80]:
# # linux bash to check the generated csv files
# cat HCAT/fanzhuan_combined/*ed.csv | wc -l
# cat HCAT/qiehuan1_combined/*ed.csv | wc -l
# cat SeqTrack/fanzhuan_combined/*ed.csv | wc -l
# cat SeqTrack/qiehuan1_combined/*ed.csv | wc -l

In [81]:
print(dataset_iou("HCAT","fanzhuan"))

  0%|          | 0/20 [00:00<?, ?it/s]

0th bbox HCAT length read: 1272
0th ground truth SeqTrack length read: 1272
bbox: (1272, 4)
gt: (1272, 4)
(1272,)
(1272,)
IOU: [1.         0.89458853 0.88896275 ... 0.89664485 0.8722387  0.89148584]
1th bbox HCAT length read: 906
1th ground truth SeqTrack length read: 906
bbox: (906, 4)
gt: (906, 4)
(906,)
(906,)
IOU: [1.         0.90381847 0.91166508 0.93378314 0.8858366  0.92121695
 0.88642958 0.89685627 0.88751187 0.87836261 0.90979016 0.9126915
 0.9081989  0.92990629 0.89084205 0.91748672 0.9057715  0.91297581
 0.9218459  0.92714019 0.92543424 0.9176824  0.91777459 0.95236077
 0.92907659 0.92071655 0.93367236 0.92259739 0.94607225 0.95389784
 0.93316008 0.9592209  0.94351139 0.95915375 0.92881152 0.93262937
 0.92269065 0.92378553 0.93614971 0.94385775 0.94883603 0.89518313
 0.92696133 0.91371202 0.94396265 0.90892539 0.93932114 0.92169779
 0.91887258 0.90639001 0.92303675 0.91205451 0.91757981 0.90947087
 0.90715167 0.90527586 0.90047773 0.91552992 0.90466904 0.90382494
 0.8803097 

 50%|█████     | 10/20 [00:00<00:00, 95.04it/s]

7th bbox HCAT length read: 884
7th ground truth SeqTrack length read: 884
bbox: (884, 4)
gt: (884, 4)
(884,)
(884,)
IOU: [1.         0.80213619 0.80747749 0.85002777 0.91356869 0.90658571
 0.90220347 0.88716875 0.905387   0.92955189 0.93870808 0.92339849
 0.91996745 0.91906697 0.93648081 0.92889924 0.93022067 0.94998415
 0.95682285 0.93422698 0.94215099 0.94787858 0.93032722 0.9491008
 0.92701593 0.93817681 0.90492624 0.95007939 0.93886454 0.95420553
 0.94534163 0.95504154 0.91784537 0.96432285 0.93938403 0.93969206
 0.92337631 0.92327911 0.92481225 0.90736191 0.91210298 0.91776506
 0.93678535 0.95350164 0.92372738 0.94351413 0.93506482 0.92203258
 0.95329605 0.92435987 0.93351457 0.89131277 0.92286626 0.91891816
 0.92743529 0.9270198  0.94057644 0.89800599 0.59336481 0.74261809
 0.93124577 0.82429376 0.83760834 0.89308049 0.84545692 0.91820436
 0.9017404  0.8412678  0.81271443 0.85210848 0.8662248  0.84872936
 0.84962914 0.81797943 0.86573324 0.88158596 0.84417117 0.86409887
 0.838890

100%|██████████| 20/20 [00:00<00:00, 90.33it/s]

(1296,)
(1296,)
IOU: [1.         0.81630049 0.8598659  ... 0.91509199 0.91309996 0.90304537]
19th bbox HCAT length read: 944
19th ground truth SeqTrack length read: 944
bbox: (944, 4)
gt: (944, 4)
(944,)
(944,)
IOU: [1.         0.88863725 0.86057356 0.83101122 0.80514387 0.8041427
 0.83078351 0.79501455 0.82238325 0.81518592 0.80793388 0.79760031
 0.76582902 0.78364862 0.75958177 0.76324801 0.74958039 0.75095396
 0.77536815 0.80203433 0.75504192 0.76163977 0.75174846 0.76689495
 0.77828346 0.75820965 0.80044633 0.81362982 0.81063732 0.83779203
 0.83761134 0.82659801 0.83883817 0.82301173 0.83961268 0.86601081
 0.85721293 0.84611044 0.85966258 0.89388929 0.91419852 0.8867769
 0.89507446 0.89852048 0.89274519 0.9018784  0.8829845  0.85546543
 0.8596891  0.85323887 0.88362116 0.87622276 0.86193921 0.85226137
 0.87420261 0.85998682 0.84228017 0.85042929 0.87023305 0.84173874
 0.83605784 0.8402706  0.8853581  0.87393998 0.84972218 0.85190239
 0.85121975 0.85870615 0.86310452 0.87152351 0.85

In [82]:
print(dataset_iou("HCAT","qiehuan1"))

  0%|          | 0/43 [00:00<?, ?it/s]

0th bbox HCAT length read: 585
0th ground truth SeqTrack length read: 585
bbox: (585, 4)
gt: (585, 4)
(585,)
(585,)
IOU: [1.         0.82569874 0.84889149 0.82582354 0.85051408 0.82888758
 0.84303392 0.85571416 0.86366649 0.85004579 0.87021577 0.85890484
 0.85080023 0.86375661 0.84857124 0.85749526 0.83832001 0.83591323
 0.89530921 0.87472509 0.89282546 0.89074152 0.86287973 0.86929246
 0.87350516 0.85626286 0.88565466 0.85597145 0.87877139 0.8998068
 0.89192606 0.86380093 0.88626998 0.88781465 0.86884142 0.88815925
 0.893359   0.89755941 0.87367702 0.88328432 0.87598573 0.90499867
 0.8776516  0.89620652 0.89707468 0.91862596 0.91368082 0.87580261
 0.89964967 0.8785168  0.89773384 0.8747698  0.88668611 0.8850817
 0.88687163 0.84315164 0.8735295  0.84703189 0.8665581  0.8348871
 0.86284931 0.82170262 0.83914649 0.84214372 0.85536803 0.84684709
 0.85760706 0.85920851 0.83269765 0.82083566 0.8300564  0.85516118
 0.83219239 0.85928006 0.82743818 0.84963409 0.83297603 0.84233329
 0.86403575

 23%|██▎       | 10/43 [00:00<00:00, 87.73it/s]

bbox: (852, 4)
gt: (852, 4)
(852,)
(852,)
IOU: [1.         0.94769797 0.95256161 0.93407221 0.92447786 0.93686411
 0.92639096 0.87538027 0.93239439 0.92712701 0.91087519 0.92631878
 0.93859484 0.92362345 0.92746322 0.94220037 0.90833176 0.91607463
 0.9399814  0.94854481 0.93293355 0.94407311 0.91124736 0.96910603
 0.9527067  0.9409385  0.95823809 0.92084719 0.96935298 0.9646802
 0.94120568 0.97180279 0.94792718 0.94846949 0.92479556 0.98972927
 0.9014599  0.90607627 0.9182867  0.92582525 0.92152809 0.9400144
 0.91136065 0.92704935 0.89265075 0.93349852 0.9213811  0.89912919
 0.96077763 0.9326794  0.93175673 0.9213535  0.93963566 0.93942559
 0.94115211 0.93591079 0.96845226 0.94470635 0.9528703  0.95252642
 0.96871521 0.94634911 0.98067909 0.95210688 0.94199513 0.97935679
 0.95945188 0.92754756 0.95890495 0.9534665  0.95086194 0.95008091
 0.9470579  0.95669292 0.9521061  0.95222691 0.94884088 0.93563185
 0.97734243 0.93756985 0.93164938 0.96672727 0.95752819 0.94819985
 0.95244761 0.926

 44%|████▍     | 19/43 [00:00<00:00, 85.06it/s]

...

26th ground truth SeqTrack length read: 512
bbox: (512, 4)
gt: (512, 4)
(512,)
(512,)
IOU: [1.         0.89281429 0.88981226 0.90793461 0.90844711 0.89210132
 0.88864004 0.87708707 0.87107067 0.85138999 0.85279027 0.85826441
 0.85486004 0.86250195 0.85942778 0.8498516  0.8721806  0.86001773
 0.84903437 0.85102553 0.86078243 0.84879004 0.87366042 0.86382506
 0.82755353 0.8503725  0.84597325 0.85337763 0.8437163  0.84239461
 0.82800931 0.84954378 0.83371374 0.82837657 0.82479601 0.84530107
 0.85508177 0.82014802 0.84563069 0.87634749 0.87862591 0.87605108
 0.86696678 0.85716358 0.87659005 0.8809988  0.86362291 0.88909825
 0.86570932 0.87335017 0.8636036  0.85668015 0.86116383 0.86128132
 0.85861843 0.81955723 0.84086198 0.8557427  0.86434242 0.83808429
 0.84525906 0.83240222 0.85488007 0.82679907 0.88612708 0.84450067
 0.83519963 0.83456455 0.83278523 0.85401165 0.83250327 0.84189537
 0.85156624 0.86736699 0.85528333 0.84134072 0.8708843  0.86811112
 0.85465601 0.89016018 0.89372476 0.87

 86%|████████▌ | 37/43 [00:00<00:00, 83.07it/s]

35th ground truth SeqTrack length read: 956
bbox: (956, 4)
gt: (956, 4)
(956,)
(956,)
IOU: [1.00000000e+00 9.62877650e-01 9.40154614e-01 9.35172028e-01
 9.37547711e-01 9.14423325e-01 9.26950174e-01 9.16457758e-01
 9.39497350e-01 8.97484984e-01 9.01750422e-01 9.25484680e-01
 9.15119628e-01 8.91188709e-01 9.26182560e-01 9.35245888e-01
 9.19887272e-01 9.03621042e-01 9.12011745e-01 9.12454467e-01
 8.92674697e-01 8.94666099e-01 9.21596294e-01 9.29156003e-01
 9.00502844e-01 9.21903050e-01 9.21856590e-01 9.13632288e-01
 9.16123901e-01 9.18056668e-01 9.28532940e-01 9.04904424e-01
 9.15331841e-01 9.26723745e-01 9.00107321e-01 9.06631531e-01
 9.24412056e-01 9.34825928e-01 8.81317143e-01 8.99610741e-01
 8.92178203e-01 8.63919164e-01 8.97002388e-01 9.07122304e-01
 8.87503001e-01 8.76054255e-01 9.09460003e-01 9.10479109e-01
 8.47617216e-01 8.94383870e-01 8.64213957e-01 8.63412272e-01
 8.69059016e-01 8.81061347e-01 8.64724540e-01 8.85653654e-01
 8.74749357e-01 8.97548697e-01 8.69177868e-01 9.1070909

100%|██████████| 43/43 [00:00<00:00, 85.29it/s]

(589,)
(589,)
IOU: [1.         0.91923009 0.92530197 0.89748647 0.87146787 0.90239585
 0.89124399 0.89807234 0.88169477 0.87526877 0.86495534 0.85168229
 0.89265353 0.84232421 0.85044628 0.84423859 0.87676709 0.85506622
 0.85425699 0.854747   0.89030827 0.90055312 0.87371856 0.88050941
 0.87091079 0.87144361 0.86823283 0.86786326 0.8755562  0.85853031
 0.83546664 0.85816956 0.82414054 0.80777919 0.83934285 0.82813378
 0.78350202 0.83384924 0.84885428 0.82094825 0.81531061 0.8092969
 0.83441707 0.75660245 0.79691788 0.78228455 0.79088719 0.78360955
 0.78654284 0.79270265 0.79713609 0.81331501 0.82056596 0.79838095
 0.81909825 0.82497474 0.78226364 0.78882257 0.81904312 0.8032339
 0.80734535 0.78842977 0.8139376  0.82901183 0.7880338  0.79952138
 0.8036942  0.79571264 0.81515282 0.8188494  0.79656536 0.80993386
 0.84248833 0.83456627 0.7756408  0.81967109 0.84375177 0.82935864
 0.84397191 0.8085266  0.81762627 0.76008727 0.80844611 0.8392581
 0.8349798  0.76558167 0.8338854  0.78289491 0

In [83]:
hcat_fanzhuan_iou = dataset_iou("HCAT","fanzhuan")
hcat_qiehuan1_iou = dataset_iou("HCAT","qiehuan1")
hcat_fanzhuan_df = pd.DataFrame(hcat_fanzhuan_iou)
hcat_qiehuan1_df = pd.DataFrame(hcat_qiehuan1_iou)


hcat_fanzhuan_df.to_csv("hcat_fanzhuan.csv")
hcat_qiehuan1_df.to_csv("hcat_qiehuan1.csv")
avg_hcat_fanzhuan = hcat_fanzhuan_df.mean()
avg_hcat_qiehuan1 = hcat_qiehuan1_df.mean()

  0%|          | 0/20 [00:00<?, ?it/s]

0th bbox HCAT length read: 1272
0th ground truth SeqTrack length read: 1272
bbox: (1272, 4)
gt: (1272, 4)
(1272,)
(1272,)
IOU: [1.         0.89458853 0.88896275 ... 0.89664485 0.8722387  0.89148584]
1th bbox HCAT length read: 906
1th ground truth SeqTrack length read: 906
bbox: (906, 4)
gt: (906, 4)
(906,)
(906,)
IOU: [1.         0.90381847 0.91166508 0.93378314 0.8858366  0.92121695
 0.88642958 0.89685627 0.88751187 0.87836261 0.90979016 0.9126915
 0.9081989  0.92990629 0.89084205 0.91748672 0.9057715  0.91297581
 0.9218459  0.92714019 0.92543424 0.9176824  0.91777459 0.95236077
 0.92907659 0.92071655 0.93367236 0.92259739 0.94607225 0.95389784
 0.93316008 0.9592209  0.94351139 0.95915375 0.92881152 0.93262937
 0.92269065 0.92378553 0.93614971 0.94385775 0.94883603 0.89518313
 0.92696133 0.91371202 0.94396265 0.90892539 0.93932114 0.92169779
 0.91887258 0.90639001 0.92303675 0.91205451 0.91757981 0.90947087
 0.90715167 0.90527586 0.90047773 0.91552992 0.90466904 0.90382494
 0.8803097 

 45%|████▌     | 9/20 [00:00<00:00, 89.91it/s]

7th bbox HCAT length read: 884
7th ground truth SeqTrack length read: 884
bbox: (884, 4)
gt: (884, 4)
(884,)
(884,)
IOU: [1.         0.80213619 0.80747749 0.85002777 0.91356869 0.90658571
 0.90220347 0.88716875 0.905387   0.92955189 0.93870808 0.92339849
 0.91996745 0.91906697 0.93648081 0.92889924 0.93022067 0.94998415
 0.95682285 0.93422698 0.94215099 0.94787858 0.93032722 0.9491008
 0.92701593 0.93817681 0.90492624 0.95007939 0.93886454 0.95420553
 0.94534163 0.95504154 0.91784537 0.96432285 0.93938403 0.93969206
 0.92337631 0.92327911 0.92481225 0.90736191 0.91210298 0.91776506
 0.93678535 0.95350164 0.92372738 0.94351413 0.93506482 0.92203258
 0.95329605 0.92435987 0.93351457 0.89131277 0.92286626 0.91891816
 0.92743529 0.9270198  0.94057644 0.89800599 0.59336481 0.74261809
 0.93124577 0.82429376 0.83760834 0.89308049 0.84545692 0.91820436
 0.9017404  0.8412678  0.81271443 0.85210848 0.8662248  0.84872936
 0.84962914 0.81797943 0.86573324 0.88158596 0.84417117 0.86409887
 0.838890

 90%|█████████ | 18/20 [00:00<00:00, 89.74it/s]

...

bbox: (566, 4)
gt: (566, 4)
(566,)
(566,)
IOU: [1.         0.17109483 0.17873345 0.16811297 0.16745265 0.1668416
 0.16915442 0.16624442 0.1691394  0.13034481 0.07482414 0.02295708
 0.         0.         0.00243162 0.01504999 0.01444679 0.020958
 0.02436384 0.03017895 0.03931436 0.03766519 0.03833037 0.04154335
 0.04155974 0.04239366 0.04603364 0.04513909 0.0536865  0.04502459
 0.04146697 0.01728956 0.         0.         0.         0.
 0.         0.27231989 0.31857708 0.0938195  0.05305824 0.68410184
 0.78716907 0.8246232  0.86617286 0.83096206 0.69044072 0.64991809
 0.33768103 0.18710561 0.24921159 0.22131587 0.22833973 0.50231369
 0.6297044  0.85045073 0.24898299 0.32308351 0.33303897 0.32879389
 0.3866496  0.70585025 0.45350883 0.47741925 0.46973892 0.48000805
 0.47242371 0.47934881 0.37129811 0.49405037 0.31763886 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.   

 84%|████████▎ | 36/43 [00:00<00:00, 81.98it/s]

35th ground truth SeqTrack length read: 956
bbox: (956, 4)
gt: (956, 4)
(956,)
(956,)
IOU: [1.00000000e+00 9.62877650e-01 9.40154614e-01 9.35172028e-01
 9.37547711e-01 9.14423325e-01 9.26950174e-01 9.16457758e-01
 9.39497350e-01 8.97484984e-01 9.01750422e-01 9.25484680e-01
 9.15119628e-01 8.91188709e-01 9.26182560e-01 9.35245888e-01
 9.19887272e-01 9.03621042e-01 9.12011745e-01 9.12454467e-01
 8.92674697e-01 8.94666099e-01 9.21596294e-01 9.29156003e-01
 9.00502844e-01 9.21903050e-01 9.21856590e-01 9.13632288e-01
 9.16123901e-01 9.18056668e-01 9.28532940e-01 9.04904424e-01
 9.15331841e-01 9.26723745e-01 9.00107321e-01 9.06631531e-01
 9.24412056e-01 9.34825928e-01 8.81317143e-01 8.99610741e-01
 8.92178203e-01 8.63919164e-01 8.97002388e-01 9.07122304e-01
 8.87503001e-01 8.76054255e-01 9.09460003e-01 9.10479109e-01
 8.47617216e-01 8.94383870e-01 8.64213957e-01 8.63412272e-01
 8.69059016e-01 8.81061347e-01 8.64724540e-01 8.85653654e-01
 8.74749357e-01 8.97548697e-01 8.69177868e-01 9.1070909

100%|██████████| 43/43 [00:00<00:00, 83.66it/s]

41th ground truth SeqTrack length read: 814
bbox: (814, 4)
gt: (814, 4)
(814,)
(814,)
IOU: [1.         0.94178496 0.92759178 0.91129914 0.92879722 0.92202168
 0.91729483 0.92539428 0.93191665 0.94222159 0.94982775 0.93081404
 0.95823366 0.9535399  0.94733209 0.89133554 0.94175417 0.92625909
 0.93046971 0.929835   0.92621636 0.90407985 0.9192597  0.94595356
 0.89699968 0.95339455 0.93134658 0.90863074 0.95557024 0.91718635
 0.96876251 0.92370867 0.9314082  0.91192739 0.91664093 0.92342288
 0.93493277 0.95592175 0.95060493 0.93532482 0.93912451 0.95366649
 0.93881987 0.94928072 0.92541991 0.92602978 0.93572339 0.91874626
 0.94469366 0.93032841 0.93571735 0.93151527 0.91743658 0.96298853
 0.9690003  0.96241437 0.94894987 0.94536622 0.96143039 0.9540846
 0.93174243 0.95956456 0.92587193 0.93397731 0.93407697 0.95888876
 0.93949613 0.93813445 0.96361977 0.95592464 0.96704207 0.97470419
 0.96119439 0.96683819 0.94673842 0.98468266 0.93156037 0.97560492
 0.9371061  0.95344037 0.9477359  0.945

In [84]:
stark_fanzhuan_iou = dataset_iou("Stark","fanzhuan")
stark_qiehuan1_iou = dataset_iou("Stark","qiehuan1")
stark_fanzhuan_df = pd.DataFrame(stark_fanzhuan_iou)
stark_qiehuan1_df = pd.DataFrame(stark_qiehuan1_iou)


stark_fanzhuan_df.to_csv("stark_fanzhuan.csv")
stark_qiehuan1_df.to_csv("stark_qiehuan1.csv")

avg_stark_fanzhuan = stark_fanzhuan_df.mean()
avg_stark_qiehuan1 = stark_qiehuan1_df.mean()


  0%|          | 0/20 [00:00<?, ?it/s]

0th bbox Stark length read: 1272
0th ground truth SeqTrack length read: 1272
bbox: (1272, 4)
gt: (1272, 4)
(1272,)
(1272,)
IOU: [1.         0.96316575 0.96428571 ... 0.9530652  0.98969072 0.94949495]
1th bbox Stark length read: 906
1th ground truth SeqTrack length read: 906
bbox: (906, 4)
gt: (906, 4)
(906,)
(906,)
IOU: [1.         0.96532696 0.9509488  0.93990192 0.94044796 0.95108359
 0.95136084 0.95149254 0.95176572 0.96296296 0.97037037 0.948386
 0.94148981 0.95216362 0.95216362 0.94565217 0.95281621 0.9530739
 0.96402878 0.94638844 0.93967647 0.95370149 0.9500394  0.94710485
 0.95406886 0.96103286 0.96103286 0.93357934 0.95735954 0.96811639
 0.94382541 0.95443019 0.9375521  0.95108696 0.94070465 0.93739801
 0.94104561 0.95802099 0.93797828 0.95168655 0.95852535 0.9625109
 0.95874946 0.9625109  0.9625109  0.93867824 0.97578816 0.97315436
 0.95244895 0.97599195 0.97608168 0.96580714 0.95936842 0.95936842
 0.9762807  0.96326246 0.96326246 0.96326246 0.95301499 0.95301499
 0.94351522 

 50%|█████     | 10/20 [00:00<00:00, 96.25it/s]

9th bbox Stark length read: 1011
9th ground truth SeqTrack length read: 1011
bbox: (1011, 4)
gt: (1011, 4)
(1011,)
(1011,)
IOU: [1.         0.76676059 0.76274344 ... 0.83535109 0.80182223 0.81013525]
10th bbox Stark length read: 845
10th ground truth SeqTrack length read: 845
bbox: (845, 4)
gt: (845, 4)
(845,)
(845,)
IOU: [1.         0.81745593 0.77976028 0.86442406 0.82895497 0.82245431
 0.86736617 0.83443709 0.7732385  0.77537027 0.82186872 0.79665298
 0.76644463 0.81474011 0.76951528 0.74929577 0.75717771 0.76707958
 0.74756873 0.74377905 0.73180539 0.76745356 0.74360348 0.75796867
 0.71186441 0.76038023 0.73025166 0.74458272 0.71614583 0.7752988
 0.7704962  0.77482111 0.82070004 0.7925608  0.7513693  0.72307892
 0.7583025  0.74458503 0.77279986 0.75684259 0.77114954 0.73339988
 0.76684175 0.72244016 0.67790875 0.68046601 0.68706741 0.64954376
 0.69517421 0.6336109  0.70032211 0.66610745 0.73025249 0.69441616
 0.80042463 0.64694456 0.73063128 0.70066574 0.76443805 0.71555556
 0.7621

100%|██████████| 20/20 [00:00<00:00, 92.20it/s]


...

27th bbox Stark length read: 557
27th ground truth SeqTrack length read: 557
bbox: (557, 4)
gt: (557, 4)
(557,)
(557,)
IOU: [1.         0.95558572 0.9607454  0.97050814 0.96215514 0.9563396
 0.96087629 0.95645705 0.9308823  0.92078538 0.93005181 0.9120086
 0.94558707 0.93479726 0.94405335 0.9508727  0.93735717 0.94566702
 0.93552091 0.9402895  0.94491682 0.94663973 0.93792466 0.94697844
 0.94743898 0.94315984 0.95068632 0.94233358 0.95229707 0.95714211
 0.97140475 0.96639251 0.96222308 0.95220018 0.95047006 0.95768512
 0.95167975 0.94801776 0.93725555 0.91640091 0.92968164 0.95241065
 0.93517109 0.94429721 0.93367418 0.93289455 0.93895951 0.96401509
 0.94522245 0.96669653 0.96239432 0.95459756 0.94523242 0.95444534
 0.94940662 0.95122257 0.93930239 0.96903782 0.94633955 0.96798714
 0.94945873 0.97657033 0.96286299 0.9609072  0.95576571 0.96424946
 0.96836555 0.96335677 0.95940355 0.95956722 0.96232379 0.95747142
 0.93277666 0.94623825 0.93450771 0.94302011 0.93308515 0.94029449
 0.9325

 88%|████████▊ | 38/43 [00:00<00:00, 83.07it/s]

38th bbox Stark length read: 589
38th ground truth SeqTrack length read: 589
bbox: (589, 4)
gt: (589, 4)
(589,)
(589,)
IOU: [1.         0.87178378 0.90671925 0.88199359 0.87425925 0.8783863
 0.88861689 0.85975326 0.88550129 0.85971082 0.88066294 0.86116068
 0.8995729  0.88469673 0.87339157 0.90595824 0.89773777 0.92792616
 0.88343664 0.91418199 0.90715673 0.92310707 0.90696986 0.93247974
 0.90017571 0.93823443 0.89658434 0.93131527 0.92304679 0.91331732
 0.92139164 0.89009405 0.88637648 0.87836364 0.85722534 0.86938375
 0.88511512 0.85599847 0.88245155 0.8574145  0.86890169 0.85164894
 0.79143097 0.78423615 0.85598576 0.79839397 0.77317806 0.85493077
 0.79882162 0.80533263 0.82501976 0.84315551 0.86515629 0.8377907
 0.83568985 0.84471108 0.80883079 0.86208659 0.84987212 0.82537264
 0.83730337 0.82622391 0.85822347 0.81818182 0.79925323 0.83336132
 0.84640077 0.89473815 0.87140377 0.87676685 0.87015268 0.87811997
 0.87090225 0.87774797 0.85787502 0.81594924 0.81421132 0.84184641
 0.8336

100%|██████████| 43/43 [00:00<00:00, 84.93it/s]

42th bbox Stark length read: 608
42th ground truth SeqTrack length read: 608
bbox: (608, 4)
gt: (608, 4)
(608,)
(608,)
IOU: [1.         0.93208289 0.91474073 0.93513431 0.92817883 0.9244233
 0.90080504 0.90192434 0.85964303 0.88288095 0.90950161 0.89054248
 0.9115124  0.93537406 0.94775972 0.93239828 0.93266205 0.92350024
 0.91974851 0.90879522 0.87768982 0.88917426 0.90209848 0.83638964
 0.86902344 0.82587857 0.82999787 0.82711408 0.81966126 0.88541458
 0.84949091 0.81184634 0.87154378 0.85151967 0.90232233 0.89577724
 0.83085929 0.7942683  0.86590751 0.85055323 0.86846528 0.84342405
 0.8337228  0.89445955 0.89599396 0.84960264 0.91016825 0.85976886
 0.88150432 0.84807258 0.86399448 0.86090023 0.86103005 0.84831631
 0.86051019 0.83055684 0.85751396 0.94045293 0.92376115 0.86272005
 0.84278998 0.8541405  0.83707944 0.840237   0.86060709 0.8864882
 0.78806771 0.82057823 0.87741176 0.83675597 0.85195772 0.83441351
 0.81474864 0.81962175 0.81670563 0.83288175 0.8878388  0.88959497
 0.8936

In [89]:
stark_lightning_fanzhuan_iou = dataset_iou("Stark_lightning","fanzhuan")
stark_lightning_qiehuan1_iou = dataset_iou("Stark_lightning","qiehuan1")
stark_lightning_fanzhuan_df = pd.DataFrame(stark_lightning_fanzhuan_iou)
stark_lightning_qiehuan1_df = pd.DataFrame(stark_lightning_qiehuan1_iou)


stark_lightning_fanzhuan_df.to_csv("stark_lightning_fanzhuan.csv")
stark_lightning_qiehuan1_df.to_csv("stark_lightning_qiehuan1.csv")

avg_stark_lightning_fanzhuan = stark_lightning_fanzhuan_df.mean()
avg_stark_lightning_qiehuan1 = stark_lightning_qiehuan1_df.mean()

 37%|███▋      | 7/19 [00:00<00:00, 64.77it/s]

0th bbox Stark_lightning length read: 1272
0th ground truth SeqTrack length read: 1272
bbox: (1272, 4)
gt: (1272, 4)
(1272,)
(1272,)
IOU: [1.         0.95876031 0.92816913 ... 0.87831227 0.88373183 0.8958803 ]
1th bbox Stark_lightning length read: 906
1th ground truth SeqTrack length read: 906
bbox: (906, 4)
gt: (906, 4)
(906,)
(906,)
IOU: [1.         0.93772827 0.92685095 0.91164095 0.91858038 0.90150538
 0.8956743  0.91962617 0.87586434 0.91193395 0.92096506 0.92490644
 0.94461305 0.90751398 0.89290978 0.9034931  0.88762969 0.89567103
 0.91046943 0.88989227 0.92373894 0.91935612 0.89344072 0.90764925
 0.89691363 0.89691363 0.88878591 0.85896463 0.8531288  0.89281845
 0.87589334 0.87540785 0.8751318  0.86654589 0.87106447 0.9047976
 0.86219699 0.87728636 0.88893799 0.89085285 0.87938931 0.8788143
 0.87009063 0.86741768 0.84608758 0.85911443 0.86294896 0.89557624
 0.87040454 0.84400819 0.89100199 0.83074481 0.86788914 0.85642946
 0.87657745 0.86083362 0.84081042 0.86159817 0.85661131 0

100%|██████████| 19/19 [00:00<00:00, 67.86it/s]


[1.         0.88232702 0.87870538 0.87508681 0.83878879 0.8368075
 0.83747104 0.82550998 0.8566905  0.84663542 0.83443709 0.8365013
 0.82894737 0.84677025 0.8342003  0.83041566 0.82894851 0.84124274
 0.83900259 0.82477391 0.84623542 0.80496416 0.83910526 0.82364028
 0.8291633  0.80391897 0.83128793 0.81746326 0.82042272 0.81182173
 0.80312704 0.84674121 0.79156328 0.81183394 0.81455474 0.81756543
 0.84413391 0.8392566  0.84178158 0.84608177 0.82884894 0.82768978
 0.82367102 0.84775535 0.85137472 0.85192523 0.8749241  0.85134981
 0.83892866 0.83601222 0.83194943 0.84273751 0.85113715 0.8544046
 0.82914744 0.83771026 0.87244438 0.85425523 0.87731882 0.85046132
 0.86748566 0.86513988 0.87955244 0.86803874 0.86644035 0.87421384
 0.87050076 0.86054422 0.84054159 0.8425071  0.83189172 0.83495942
 0.83126749 0.87431115 0.80785953 0.73067531 0.87409151 0.75721095
 0.79018006 0.73291103 0.80184984 0.71869783 0.85876425 0.74151557
 0.80878632 0.73324905 0.81761732 0.73586207 0.82580685 0.7578075

 19%|█▉        | 8/42 [00:00<00:00, 72.73it/s]

0th bbox Stark_lightning length read: 585
0th ground truth SeqTrack length read: 585
bbox: (585, 4)
gt: (585, 4)
(585,)
(585,)
IOU: [1.00000000e+00 9.64468357e-01 9.30502713e-01 9.21865305e-01
 9.11554995e-01 9.16972753e-01 8.96040234e-01 8.89736369e-01
 8.85321340e-01 9.05674147e-01 8.81025500e-01 9.10319784e-01
 8.68655981e-01 8.51348820e-01 8.60896083e-01 8.44296218e-01
 8.74687262e-01 8.46874350e-01 8.42968137e-01 8.26162791e-01
 8.32904922e-01 8.31449439e-01 8.23697421e-01 8.24068253e-01
 8.21716989e-01 8.26987952e-01 8.15857826e-01 8.24345227e-01
 8.18766930e-01 8.18304209e-01 8.36985951e-01 8.43324789e-01
 8.47633136e-01 8.61988693e-01 8.33333333e-01 8.50507605e-01
 8.39749096e-01 8.38377123e-01 8.58315765e-01 8.35859593e-01
 8.35137746e-01 8.47260972e-01 8.28663800e-01 8.52236382e-01
 8.35320135e-01 8.38448103e-01 8.36097138e-01 8.57048663e-01
 8.30883053e-01 8.64680591e-01 8.22517461e-01 8.23499882e-01
 8.30112728e-01 8.68306334e-01 8.64949709e-01 8.82448762e-01
 9.01319329e-0

 60%|█████▉    | 25/42 [00:00<00:00, 75.67it/s]

14th ground truth SeqTrack length read: 920
bbox: (920, 4)
gt: (920, 4)
(920,)
(920,)
IOU: [1.         0.87111367 0.85893268 0.8581599  0.86011303 0.85084484
 0.84821785 0.84678239 0.8584445  0.84748728 0.86185515 0.86535506
 0.87947417 0.85728462 0.87954282 0.86682134 0.85809048 0.85591061
 0.84098854 0.84960126 0.86222271 0.84530621 0.83585207 0.85632832
 0.84144781 0.85984585 0.84713049 0.84901152 0.85208706 0.84385667
 0.86681683 0.85920882 0.86335161 0.85045164 0.84088759 0.83287282
 0.83217097 0.83163719 0.84180441 0.85857825 0.84849603 0.86261024
 0.86008102 0.84832659 0.85197906 0.86077458 0.83875988 0.86068604
 0.83010673 0.85818433 0.84677871 0.85174125 0.82436943 0.84676013
 0.84248962 0.84301404 0.84181044 0.80926123 0.83544457 0.8023254
 0.86884971 0.87457694 0.84995973 0.85983366 0.86238612 0.84239904
 0.86232738 0.87177111 0.83968099 0.84052344 0.85267869 0.85493789
 0.83338763 0.79702613 0.74231206 0.7569386  0.81712222 0.84742489
 0.84112702 0.86108258 0.85264464 0.845

100%|██████████| 42/42 [00:00<00:00, 80.60it/s]

(802,)
(802,)
IOU: [1.         0.94594595 0.90118577 0.22024793 0.6055794  0.53478233
 0.55579207 0.80968206 0.69959411 0.80953722 0.70250744 0.09823809
 0.         0.061396   0.11273157 0.11397239 0.15530154 0.16141239
 0.24197466 0.62484095 0.49835773 0.37308556 0.48049472 0.55748457
 0.48888037 0.49637884 0.48153329 0.18580049 0.32325628 0.29647528
 0.16981919 0.32579922 0.29117924 0.24604375 0.4920258  0.04859216
 0.2736921  0.09526328 0.02911217 0.35017165 0.06065825 0.21634898
 0.0918035  0.17925179 0.57320113 0.43229418 0.24143388 0.35576607
 0.25359369 0.17409317 0.0321615  0.47640082 0.20702875 0.26644313
 0.25496952 0.26483816 0.19363933 0.385053   0.24917667 0.23882033
 0.28478964 0.23078068 0.41399055 0.40537462 0.35500079 0.46356208
 0.36165441 0.28566503 0.26587313 0.22086964 0.36029853 0.52496264
 0.19380715 0.07483141 0.0417843  0.28072675 0.23449765 0.22677941
 0.06276257 0.28830207 0.2395327  0.40182316 0.29635294 0.0355318
 0.57491208 0.1861182  0.29168958 0.01467789

In [90]:
print(avg_hcat_fanzhuan)
print(avg_hcat_qiehuan1)
print(avg_stark_fanzhuan)
print(avg_stark_qiehuan1)
print(avg_stark_lightning_fanzhuan)
print(avg_stark_lightning_qiehuan1)

0    0.837386
dtype: float64
0    0.661019
dtype: float64
0    0.898746
dtype: float64
0    0.720844
dtype: float64
0    0.814202
dtype: float64
0    0.681509
dtype: float64
